In [10]:
import csv
import tensorflow as tf
from PIL import Image

data = {1:0, 2:0, 3:0, 4:0}
digit_1, digit_2, digit_3, digit_4 = [], [], [], []
digit = [digit_1, digit_2, digit_3, digit_4]

In [11]:
def Data_Scale():
    csv_reader = csv.reader(open('data/captcha/labels/labels.csv', encoding='utf-8'))
    for row in csv_reader:
        if int(row[1])//1000:
            data[4] += 1
            digit_4.append([row[0],row[1]])
        elif int(row[1])//100:
            data[3] += 1
            digit_3.append([row[0],row[1]])
        elif int(row[1])//10:
            data[2] += 1
            digit_2.append([row[0],row[1]])
        elif int(row[1])//1:
            data[1] += 1
            digit_1.append([row[0],row[1]])
    sumdata = data[1]+data[2]+data[3]+data[4]
    # 生成txt文件
    with open('Data_Scale.txt',  'w+') as f:
        f.write("数据集总数为%d" %sumdata+"\n")
        for i in range(4):
            f.write("%d位验证码数量为%d，比例占总数的%.4f" 
              %(i+1,data[i+1],data[i+1]/sumdata)+"\n")
    print('writing...ok\n')
    # 展示生成txt内容
    file_object = open('Data_Scale.txt')
    for line in file_object:
        print(line)
Data_Scale()

writing...ok

数据集总数为40000

1位验证码数量为9844，比例占总数的0.2461

2位验证码数量为5908，比例占总数的0.1477

3位验证码数量为16183，比例占总数的0.4046

4位验证码数量为8065，比例占总数的0.2016



In [12]:
# 存储成tf.records,训练集每个文件有4720个,共四个文件,验证集和测试集每个各有2360个,各一个文件
def pro_tfrecord(data_type='train'):
    num = 5900
    loopnum = 1
    count = 0
    if data_type == 'train':
        num = num * 0.8
        loopnum = 4
    elif data_type == 'validation':
        num = num * 0.1
    elif data_type == 'test':
        num = num * 0.1
    for j in range(loopnum):
        a = j+1
        with tf.python_io.TFRecordWriter(
            'data/' + data_type + str(a) + '.tfrecords') as writer:
            for z in range(4):
                if data_type == 'train' and count == int(num):
                    continue
                if a != 1:
                    a -= 1
                    continue
                for i in range(int(num)):
                    img = Image.open(digit[z][i][0])
                    img1 = img.resize((56,40))
                    # 转灰度图像
                    image = img1.convert("L")
                    img_raw = image.tobytes()
                    example = tf.train.Example(features=tf.train.Features(feature={
                        "label": tf.train.Feature(
                            int64_list=tf.train.Int64List(value=[int(digit[z][i][1])])),
                        'img_raw': tf.train.Feature(
                            bytes_list=tf.train.BytesList(value=[img_raw]))
                    }))
                    writer.write(example.SerializeToString())
                    count += 1
                del digit[z][0:int(num)]
        count = 0
        print(data_type+str(j+1)+'.tfrecords is producing...done')
pro_tfrecord('train')
pro_tfrecord('validation')
pro_tfrecord('test')

train1.tfrecords is producing...done
train2.tfrecords is producing...done
train3.tfrecords is producing...done
train4.tfrecords is producing...done
validation1.tfrecords is producing...done
test1.tfrecords is producing...done
